In [1]:
import os

In [2]:
%pwd

'/home/sushil/ml-programming/mlops_end_to_end/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/sushil/ml-programming/mlops_end_to_end'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [8]:
import os
from mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd

[2024-11-14 17:27:56,032: INFO: utils: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.]
[2024-11-14 17:27:56,033: INFO: utils: NumExpr defaulting to 8 threads.]


In [9]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    
    ## Note: You can add different data transformation techniques such as Scaler, PCA and all
    #You can perform all kinds of EDA in ML cycle here before passing this data to the model

    # I am only adding train_test_spliting cz this data is already cleaned up

    def data_cleanup(self):
        data = pd.read_csv(self.config.data_path)
        data.drop(columns=['type'],axis=1,inplace=True)
        data.dropna(inplace=True)
        print(data.info())
        return data


    def train_test_spliting(self):
        data = self.data_cleanup()



        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)
        

In [10]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2024-11-14 17:28:02,948: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-11-14 17:28:02,950: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-14 17:28:02,952: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-11-14 17:28:02,954: INFO: common: created directory at: artifacts]
[2024-11-14 17:28:02,954: INFO: common: created directory at: artifacts/data_transformation]
<class 'pandas.core.frame.DataFrame'>
Index: 6463 entries, 0 to 6496
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         6463 non-null   float64
 1   volatile acidity      6463 non-null   float64
 2   citric acid           6463 non-null   float64
 3   residual sugar        6463 non-null   float64
 4   chlorides             6463 non-null   float64
 5   free sulfur dioxide   6463 non-null   float64
 6   total sulfur dioxide  6463 non-null   float64
 7   density  